In [401]:
import notebookimport

In [402]:
if __name__=='__main__':
    rebase='overallleader'
    rebase=''#'PAD'

In [403]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")
sp = __import__("Charts - Stage Progress")


In [404]:
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        
    grammar = tracery.Grammar(rules)
        
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [405]:
#need to get db builder -
#eg http://localhost:8888/notebooks/Documents/code/github%20forks/WRC_sketches/doodles/WRC%20API%202018%20Base.ipynb

In [406]:
import os
import sqlite3
import pandas as pd
import pytablewriter
import six

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='portugal18.db'
    conn = sqlite3.connect(dbname)
    rally='Portugal'

In [407]:
#rally='France'

In [408]:
rc='RC1'
year=2018
#ss='SS4'

In [409]:
'''year=2018
rc='RC2'
ss='SS4'


typ='stage_times_stage' #stage_times_stage stage_times_overall
typ='stage_times_overall'''

"year=2018\nrc='RC2'\nss='SS4'\n\n\ntyp='stage_times_stage' #stage_times_stage stage_times_overall\ntyp='stage_times_overall"

In [410]:
stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')

#The getEnrichedStageRank query filters down to a class - default is RC1
stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')

stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')

In [411]:
stagerank_stage.iloc[10]

diffFirst                     00:00:02.4000000
diffFirstMs                               2400
diffPrev                      00:00:00.6000000
diffPrevMs                                 600
elapsedDuration               00:02:36.7000000
elapsedDurationMs                       156700
entryId                                   2108
position                                    11
source                                 Default
stageId                                    462
stageTimeId                              18304
status                               Completed
class                                      RC1
code                                       SS1
distance                                  3.36
name                                   Lousada
snum                                         1
driver.code                                PAD
entrant.name           HYUNDAI SHELL MOBIS WRT
classrank                                   11
gainedClassPos                           False
gainedClassLe

In [412]:
import inflect
p = inflect.engine()

def timedelta_to_text(td): 
    if not td: return
    c=pd.to_timedelta(td).components
    txt=[]
    if c.hours>0:
        txt_.append(p.number_to_words(c.hours)+p.plural('hours',c.hours))
    if c.minutes>0:
        txt.append('{} {}'.format(p.number_to_words(c.minutes),p.plural('minute',c.minutes)))
    if c.seconds>0:
        seconds=c.seconds
        if c.milliseconds>0:
            seconds=seconds+c.milliseconds/1000
        txt.append('{} {}'.format(p.number_to_words(seconds),p.plural('second',seconds)))
    return p.join(txt, final_sep="and")

def timedelta_to_numtext(td):
    if not td: return
    c=pd.to_timedelta(td).components
    txt=[]
    if c.hours>0:
        txt.append('{}{}'.format(c.hours,p.plural('hr',c.hours)))
    if c.minutes>0:
        txt.append('{}m'.format(c.minutes))
    if c.seconds>0:
        seconds=c.seconds
        if c.milliseconds>0:
            seconds=round(seconds+c.milliseconds/1000, 2)
        txt.append('{}s'.format(seconds))

    return p.join(txt,conj='')

In [413]:
_code='SS5'
typ=stagerank_stage
typ['elapsedDuration_txt'] =typ['elapsedDuration'].apply(timedelta_to_numtext)
typ['diffFirst_txt'] =typ['diffFirst'].apply(timedelta_to_numtext)
_srs=typ[(typ['code']==_code) & (typ['position']<=3)][['code','position','winsinarow','elapsedDuration',
                                                    'elapsedDuration_txt', 'stagewin','diffFirst_txt',
                                                                                         'stagewincount',
                                                                                         'driver.code']]
_srs.columns=[c.replace('.','') for c in _srs.columns]
_srs.to_dict(orient='record')

[{'code': 'SS5',
  'position': 1.0,
  'winsinarow': 1,
  'elapsedDuration': '00:15:31.4000000',
  'elapsedDuration_txt': '15m 31.4s',
  'stagewin': True,
  'diffFirst_txt': '',
  'stagewincount': 2.0,
  'drivercode': 'MEE'},
 {'code': 'SS5',
  'position': 2.0,
  'winsinarow': 0,
  'elapsedDuration': '00:15:32.2000000',
  'elapsedDuration_txt': '15m 32.2s',
  'stagewin': False,
  'diffFirst_txt': '',
  'stagewincount': 0.0,
  'drivercode': 'OST'},
 {'code': 'SS5',
  'position': 3.0,
  'winsinarow': 0,
  'elapsedDuration': '00:15:32.8000000',
  'elapsedDuration_txt': '15m 32.8s',
  'stagewin': False,
  'diffFirst_txt': '1.4s',
  'stagewincount': 0.0,
  'drivercode': 'SUN'}]

In [414]:
rules = {'onetwo_':'[branch:#_onetwo_#]#branch##condition#',
         '_onetwo_':'#stagewin._branch(True,stagewinner_,second)# ',
         'y':'[switch:#sval#]#switch##condition#',
         'yy':'[switch:#position.int_to_words.ordinal._prefix(stage)._switch#]#switch##condition#',
         'sval':'#position.int_to_words.ordinal._prefix(stage)._switch#',
         'stagefirst':'in first',
         'stagethird':'took third',
         'stagewinner_': "#drivercode# #won_# stage #code# #inwith_# a time of #elapsedDuration_txt#.",
         'won_':['won', 'took'],
         'inwith_':['in', 'with'],
        'stagesecond':'#drivercode# took second, #diffFirst_txt# behind.',
        'z':'#position._gte(1,stagewinner)#',
        'toptwo_':'[switch:#_toptwo_#]#switch##condition#','_toptwo_':'#position._gte(2,onetwo_)#',
        'null':''}

#dd.columns=[c.replace('.','') for c in dd.columns]
#dd['report'] = dd.apply(lambda row: pandas_row_mapper(row, rules, "#origin#"), axis=1)
#for k in row:
#    rules[k] = str(row[k])

#grammar = tracery.Grammar(rules)
#grammar.add_modifiers(inflect_english)
#grammar.flatten('stagewinner')
_srs.apply(lambda row: pandas_row_mapper(row, rules, "#y#",[base_english,inflect_english,pytracery_logic]), axis=1)

57                     in first
67    OST took second,  behind.
69                   took third
dtype: object

In [415]:
typ=stagerank_overall
_sro=typ[(typ['code']==_code) & (typ['stagewin'])][['code','position','winsinarow',
                                                                                         'stagewin',
                                                                                         'stagewincount',
                                                                                         'retainedOverallLead',
                                                                                         'driver.code']]

_sro.columns=[c.replace('.','') for c in _sro.columns]
_sro.to_dict(orient='record')

[{'code': 'SS5',
  'position': 1,
  'winsinarow': 1,
  'stagewin': True,
  'stagewincount': 2.0,
  'retainedOverallLead': False,
  'drivercode': 'MEE'}]

In [416]:
basepath = 'report'
imgdir = 'images'
imgdirfull = '{}/{}'.format(basepath, imgdir)


In [417]:
sections = stageresults.groupby('section')

#Generate list from group: stageresults.groupby('section')['code'].apply(list)

#Generate group then iterate through groups
for key in sections.groups.keys():
    print(key, sections.get_group(key)['code'].tolist())


Section 1 ['SS1']
Section 2 ['SS2', 'SS3', 'SS4']
Section 3 ['SS5', 'SS6', 'SS7', 'SS8', 'SS9']
Section 4 ['SS10', 'SS11', 'SS12']
Section 5 ['SS13', 'SS14', 'SS15']
Section 6 ['SS16', 'SS17', 'SS18', 'SS19']
Section 7 ['SS20']


In [418]:
sectionREADME_base = '''\n### {section} Report
'''

## Section Reports

Can these be limited to e.g. just RC1?

TO DO - need something if the section is a single stage - not s split chart, just a spot chart?

In [419]:
cols=['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'elapsedDuration',
       'elapsedDurationMs', 'entryId', 'position', 'source', 'stageId',
       'stageTimeId', 'status', 'class', 'code', 'distance', 'name', 'snum',
       'driver.code', 'entrant.name', 'classrank', 'gainedClassPos',
       'gainedClassLead', 'classPosDiff', 'lostClassLead', 'retainedClassLead',
       'gainedOverallPos', 'gainedOverallLead', 'overallPosDiff',
       'lostOverallLead', 'retainedOverallLead', 'stagewin', 'stagewincount',
       'winsinarow', 'gainedTime']
cols=['driver.code','entrant.name','elapsedDuration','position','classrank' ,'diffFirst', 'diffPrev']

#stagerank_stage[stagerank_stage['code']=='SS9'][cols].sort_values('position')

In [420]:
q='''
    SELECT itc.code, itc.status
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND itc.code LIKE "SS%" 
    '''.format(rally=rally)
stage_status=pd.read_sql(q,conn).set_index('code').to_dict(orient='index')
stage_status

{'SS11': {'status': 'ToRun'},
 'SS12': {'status': 'ToRun'},
 'SS13': {'status': 'ToRun'},
 'SS14': {'status': 'ToRun'},
 'SS15': {'status': 'ToRun'},
 'SS16': {'status': 'ToRun'},
 'SS17': {'status': 'ToRun'},
 'SS18': {'status': 'ToRun'},
 'SS19': {'status': 'ToRun'},
 'SS10': {'status': 'ToRun'},
 'SS1': {'status': 'Completed'},
 'SS2': {'status': 'Completed'},
 'SS3': {'status': 'Completed'},
 'SS4': {'status': 'Completed'},
 'SS20': {'status': 'ToRun'},
 'SS9': {'status': 'Running'},
 'SS5': {'status': 'Completed'},
 'SS8': {'status': 'Running'},
 'SS7': {'status': 'Interrupted'},
 'SS6': {'status': 'Completed'}}

In [421]:
%%capture
if not os.path.exists(imgdirfull):
    os.makedirs(imgdirfull)

def get_stage_results_table(stagerank_stage,ss):
    writer = pytablewriter.MarkdownTableWriter()
    writer.stream = six.StringIO()
    tmp_df = stagerank_stage[stagerank_stage['code']==ss][cols].sort_values('position')
    tmp_df.columns=['Driver','Team','Elapsed Duration','Position','Class Rank' ,'diffFirst', 'diffPrev']
    #Drop drivers without a position
    writer.from_dataframe(tmp_df.dropna(subset=['Position']))
    writer.write_table()
    return '\n\n'+writer.stream.getvalue()

    
def sectionRankingChart(data, title, sectionREADME, key, sectionStages, fn_core, deltalabels=True, elaborate=True):
    data = sr.stageOverallEnrichers(data)
    ss = data[data['code'].isin(sectionStages)].copy()
    ss = sr.stageLegEnrichers(ss)
    if ss.empty: return sectionREADME
    fn='{}/{}'.format(imgdirfull, fn_core)

    fig, ax = sr.plotStageProgressionChart( ss, linecolor='lightgrey' , deltalabels=deltalabels, elaborate=elaborate,
                                        progress=True, stageAnnotate=False, filename=fn, title = title); 
    if fig is not None:
        fn='{}/{}'.format(imgdir, fn_core)
        sectionREADME = '''\n\n{s}{key}\n\n![]({img})\n'''.format(s=sectionREADME, key=key,img=fn)
    return sectionREADME


def sectionStagesReport(sections, stagerank_overall, stagerank_stage, basepath, rebase='overall'):
    dn ='_'+rebase if rebase and 'overall' not in rebase else ''
        
    for key in sections.groups.keys():
        #path = '{}/{}'.format(basepath,key)
        #if not os.path.exists(path):
        #    os.makedirs(path)

        out_file_name = '{bp}/{key}_report{dn}.md'.format(bp=basepath, key=key, dn=dn)

        sectionStages = sections.get_group(key)['code'].tolist()
        print(sectionStages)
        #Check section stages - if TORUN, then break
        
        sectionREADME = sectionREADME_base.format(section=key)
        
        #Stage rank chart
        fn_core='spchart_stage_{}.png'.format( key)
        title = '{} - Stage Ranking'.format(key)

        sectionREADME = sectionRankingChart(stagerank_stage, title, sectionREADME, key, sectionStages, fn_core, deltalabels=False, elaborate=False)

        #Overall rally ranking chart
        fn_core='spchart_overall_{}.png'.format( key)
        title = '{} - Overall Rally Ranking Evolution'.format(key)
        sectionREADME = sectionRankingChart(stagerank_overall, title, sectionREADME, key, sectionStages, fn_core, deltalabels=True, elaborate=True )

        #Display results as table
        # TO DO - Need to do something about stage vs rally overall result?
        for stage in sectionStages:
            if stage_status[stage]['status']=='ToRun': break
            print(stage)

            #writer = pytablewriter.MarkdownTableWriter()
            #writer.stream = six.StringIO()
            #tmp_df = stagerank_stage[stagerank_stage['code']==stage][cols].sort_values('position')
            #tmp_df.columns=['Driver','Team','Elapsed Duration','Position','Class Rank' ,'diffFirst', 'diffPrev']
            #writer.from_dataframe(tmp_df)
            #writer.write_table()
            stage_result_table = get_stage_results_table(stagerank_stage,stage)
            sectionREADME = '{}\n\n##Stage Result - {}\n\n{}\n\n'.format(sectionREADME, stage, stage_result_table)


        with open(out_file_name, 'a') as out_file:
            out_file.write(sectionREADME)
            
            
sectionStagesReport(sections, stagerank_overall, stagerank_stage, basepath,rebase=rebase)


### Stage progression chart

In [422]:
%%capture
fn='{}/spchart_full.png'.format(imgdirfull)
fig, ax = sr.plotStageProgressionChart( stagerank_overall, linecolor='lightgrey' , deltalabels=True, 
                                        progress=True, stageAnnotate=False, filename=fn);

README='''\n![]({})'''.format('{}/spchart_full.png'.format(imgdir))
with open('{}/README.md'.format(basepath), 'a') as out_file:
    out_file.write(README)

In [423]:
!ls report

README.md           SS7_report.md       Section 4_report.md entries_RC3.md
SS1_report.md       SS8_report.md       Section 5_report.md entries_RC4.md
SS2_report.md       SS9_report.md       Section 6_report.md images
SS3_report.md       SUMMARY.md          Section 7_report.md maps
SS4_report.md       Section 1_report.md docs
SS5_report.md       Section 2_report.md entries_RC1.md
SS6_report.md       Section 3_report.md entries_RC2.md


## Stage Reports

In [424]:
#overallleader = stagerank_overall[stagerank_overall['position']==1].set_index('code')[['driver.code']].to_dict(orient='dict')['driver.code']
#overallleader

In [425]:
sections.groups.keys()

dict_keys(['Section 1', 'Section 2', 'Section 3', 'Section 4', 'Section 5', 'Section 6', 'Section 7'])

In [426]:
#Look for maps
maps={}
for i in os.listdir('report/maps'):
    maps[i] = i.split('.')[0].replace('0','').replace('-','-SS').replace('SSS','SS')
smaps = {}
for m in maps:
    for i in maps[m].split('-'):
        smaps[i] = m
smaps

{'SS18': 'SS18.png',
 'SS3': 'SS03-06.png',
 'SS6': 'SS03-06.png',
 'SS16': 'SS16-19.png',
 'SS19': 'SS16-19.png',
 'SS8': 'SS08-09.png',
 'SS9': 'SS08-09.png',
 'SS11': 'SS11-14.png',
 'SS14': 'SS11-14.png',
 'SS4': 'SS04-07.png',
 'SS7': 'SS04-07.png',
 'SS2': 'SS17-20.png',
 'SS5': 'SS02-05.png',
 'SS12': 'SS12-15.png',
 'SS15': 'SS12-15.png',
 'SS1': 'SSS1.png',
 'SS13': 'SS10-13.png',
 'SS17': 'SS17-20.png'}

In [427]:
import matplotlib.pyplot as plt
import unidecode

stagecodes = stageresults['code']

def stageReport(stagerank_overall,stagerank_stage, stagecodes, stage_status, rebase='overallleader'  ):
    drivercodes = stagerank_overall['driver.code'].unique()
    links=[]
    overallleader = stagerank_overall[stagerank_overall['position']==1].set_index('code')[['driver.code']].to_dict(orient='dict')['driver.code']
    for ss in stagecodes:
        #Check section stages - if TORUN, then break
        if stage_status[ss]['status']=='ToRun': break

        print(ss)
        
        dn=''
        invert_colours=False
        if rebase=='overallleader':
            dc = overallleader[ss]
        elif rebase in drivercodes:
            dc = rebase
            dn = '_'+dc
            invert_colours=True
        else:
            dc = overallleader[ss]
            
        #Should we avoid SS1?
        #SS1 stage bars are borked - disable for now?
        #if ss=='SS1': continue
        currprevstagerank = sp.getCurrPrevStageRank(conn, rally, rc, ss)
        splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
        # TO DO - SS1 stage bars are borked - disable for now?
        #Need to sort them acc. to driver who ends up first at SS1 end?
        if ss=='SS1': splitdurations=None
        with open('{}/{}_report{}.md'.format(basepath, ss, dn), 'w') as f:
            #stage map
            if ss in smaps:
                f.write('# Stage Map - {rally}, {year} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                                  ss=str(ss).replace('SS','')))
                img='maps/{}'.format(smaps[ss])
                f.write('![]({})\n'.format(img))

            #for dc in currprevstagerank['driver.code'].unique():
            
            fig, ax = sp.rebaseStageProgressBar(currprevstagerank,dc, splitdurations=splitdurations,
                                                neg='lightgreen',pos='pink',invert_colours=invert_colours,
                                     title='Stage Progress Chart - {rally}, {year} - SS{ss}'.format(year=year,
                                                                                                    rally=rally,
                                                                                                    ss=str(ss).replace('SS','')));
            img='{}/stage_report_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
            fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
            plt.close(fig)  
            f.write('# Stage Progress Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                                  dc=dc,
                                                                                  ss=str(ss).replace('SS','')))
            f.write('![]({})\n'.format(img))

            splits = ssd. dbGetSplits(conn,rally,ss,rc)
            if splits is not None and not splits.empty :
                #print(splits)
                elapseddurations=ssd.getElapsedDurations(splits)
                try:
                    if elapseddurations is not None and not elapseddurations.empty:
                        fig, ax = ssd.plotSplitOverallDelta(elapseddurations,dc,invert_colours=invert_colours)
                        img='{}/stage_report_split_delta_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
                        fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
                        plt.close(fig)  
                        f.write('# Stage Overall Split Delta Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                                              dc=dc,
                                                                                              ss=str(ss).replace('SS','')))
                        f.write('![]({})\n'.format(img))
                except:
                    print("Error with Overall Split Delta Chart {}".format(ss))

                #try:
                splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
                if splitdurations is not None and not splitdurations.empty:
                    fig, ax = ssd.plotSplitSectionDelta(splitdurations,dc,invert_colours=invert_colours)
                    img='{}/stage_report_individual_split_delta_{}_{}.png'.format(imgdir,str(ss).replace('SS',''), unidecode.unidecode(dc))
                    fig.savefig('{}/{}'.format(basepath,img))   # save the figure to file
                    plt.close(fig)  
                    f.write('# Stage Within Split Delta Chart - {rally}, {year} - {dc} - SS{ss}\n\n'.format(year=year,rally=rally,
                                                                                          dc=dc,
                                                                                          ss=str(ss).replace('SS','')))
                    f.write('![]({})\n'.format(img))
                #except:
                #    print("Error with Within Split Delta Chart {}".format(ss))
                
            stage_result_table = get_stage_results_table(stagerank_stage,ss)
            f.write(stage_result_table)


                
#stageReport(stagerank_overall, stagecodes, stage_status, rebase='overallleader'  )
#stageReport(stagerank_overall, stagecodes, stage_status, rebase='PAD'  )
stageReport(stagerank_overall, stagerank_stage, stagecodes, stage_status, rebase=  rebase)


SS1
SS2
SS3
SS4
SS5
SS6
SS7
SS8
SS9


In [428]:
ssd.dbGetSplits(conn,rally,'SS10',rc='RC1').empty
#ssd.getSplitDurationsFromSplits(conn,rally,'SS9',rc)

True

In [429]:
!pwd

/Users/ajh59/Documents/code/github forks/WRC_sketches/doodles


In [430]:
#!/Users/ajh59/anaconda3/bin/pip install pytablewriter

In [431]:
#splitdurations.head()

In [432]:
#import pytablewriter

#writer = pytablewriter.MarkdownTableWriter()
#writer.from_dataframe(splitdurations.head())
#writer.write_table()

In [433]:
#In reports directory
!(cd report/; rm -rf docs; gitbook build;  mv _book docs)

#gitbook serve

info: 7 plugins are installed 
info: 6 explicitly listed 
info: loading plugin "highlight"... OK 
info: loading plugin "search"... OK 
info: loading plugin "lunr"... OK 
info: loading plugin "sharing"... OK 
info: loading plugin "fontsettings"... OK 
info: loading plugin "theme-default"... OK 
info: found 21 pages 
info: found 42 asset files 
info: >> generation finished with success in 2.9s ! 


In [434]:
!ls report/docs

SS1_report.html       Section 1_report.html entries_RC3.html
SS2_report.html       Section 2_report.html entries_RC4.html
SS3_report.html       Section 3_report.html gitbook
SS4_report.html       Section 4_report.html images
SS5_report.html       Section 5_report.html index.html
SS6_report.html       Section 6_report.html maps
SS7_report.html       Section 7_report.html search_index.json
SS8_report.html       entries_RC1.html
SS9_report.html       entries_RC2.html
